In [188]:
import pandas as pd
import numpy as np
path = 'data/raw/train.csv'
df = pd.read_csv(path)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [189]:
test = df.isnull().sum()
_sum = test.sum()

In [190]:
test = list(100 * df.isnull().sum() / len(df))
for val in test:
    if val >= 0 and val <= 100:
        print(val)

0.0
0.0
0.0
0.0
0.0
19.865319865319865
0.0
0.0
0.0
0.0
77.10437710437711
0.2244668911335578


In [191]:
test

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 19.865319865319865,
 0.0,
 0.0,
 0.0,
 0.0,
 77.10437710437711,
 0.2244668911335578]

In [192]:
test = list(df.dtypes)
for types in test:
    if str(types) in [np.dtype("int64"), np.dtype(object), np.dtype("float64")]:
        print(str(types))

int64
int64
int64
object
object
float64
int64
int64
object
float64
object
object


In [193]:
str(test.dtypes) in 'dtype("int64")'

AttributeError: 'list' object has no attribute 'dtypes'

In [194]:
def missing_values_table_info(df):
    # Total missing values
    mis_val = df.isnull().sum()
    
    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    
    # Coumn for dtypes
    dtype = df.dtypes
    
    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent,dtype], axis=1)
    
    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values', 2:'Data Types'})
    
    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    
    # Print some summary information
#     print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
#         "There are " + str(mis_val_table_ren_columns.shape[0]) +
#             " columns that have missing values.")
    
    # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [195]:
summary = missing_values_table_info(df)
summary

,Missing Values,% of Total Values,Data Types
Cabin,687,77.1,object
Age,177,19.9,float64
Embarked,2,0.2,object


In [196]:
type(summary) == type(pd.DataFrame)

TypeError: 'numpy.dtype' object is not callable

In [197]:
def clean_df_cols_missing_values(summary_df, threshold=75):
    return summary_df[summary_df["% of Total Values"] > threshold].index.tolist()


In [240]:
cols_list = clean_df_cols_missing_values(summary, threshold=75)
if cols_list==np.array(['Cabin']):
    print('yes')

yes


In [251]:
cols_list = clean_df_cols_missing_values(summary, threshold=75)
if cols_list==['Cabin']:
    print('yes')

yes


In [249]:
cols_list = clean_df_cols_missing_values(summary, threshold=0)
if cols_list==['Cabin', 'Age', 'Embarked']:
    print('yes')

yes


In [248]:
cols_list = clean_df_cols_missing_values(summary, threshold=10)
if cols_list==['Cabin', 'Age']:
    print('yes')

yes


In [252]:
cols_list = clean_df_cols_missing_values(summary, threshold=80)
if cols_list==[]:
    print('yes')

yes


In [420]:
def drop_cols(df, cols):
    return df.drop(columns=cols)

In [421]:
df = drop_cols(df, cols_list)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [422]:
summary_df2 = missing_values_table_info(clean_df, raw=False)
summary_df2 

Your selected dataframe has 12 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values,Data Types
Age,177,19.9,float64
Embarked,2,0.2,object


In [423]:
def cols_to_impute(df, _min=1, _max=20):
    min_cols_to_impute = df[df["% of Total Values"] <= _min].index.tolist()
    max_cols_to_impute = df[df["% of Total Values"] <= _max].index.tolist()
    return list(set(min_cols_to_impute ) and set(max_cols_to_impute))

In [424]:
list1=cols_to_impute(summary_df2)
list1

['Age', 'Embarked']

In [425]:
def filter_by_dtypes(df, lst):
    nums, others = [], []
    dtypes_list = df[lst].dtypes.tolist() 
    for col, types in zip(lst, dtypes_list):
        if types in ['float', 'int']:
            nums.append(col)
        else:
            others.append(col)
    return nums, others

In [426]:
to_impute, to_remove = filter_by_dtypes(df, list1)

In [427]:
to_remove

['Embarked']

In [428]:
def imput(df, cols):
    if cols:
        for col in cols:
            value = df[col].mean()
            df[col].fillna(value, inplace=True)
    return df

In [429]:
df = imput(df, to_impute)

In [430]:
to_remove

['Embarked']

In [431]:
null_data = df[df.isnull().any(axis=1)]
null_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,NaN


In [432]:
def clean_df_rows_missing_values(df):
    df1 = df.dropna()
    return df1

In [433]:
df = clean_df_rows_missing_values(df)

In [434]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [435]:
test = missing_values_table_info(df, raw=False)
test

Your selected dataframe has 11 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values,Data Types


In [440]:
df = pd.read_csv('../data/clean/clean_train.csv')

In [441]:
# 

In [442]:
df.head()

,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
